<a href="https://colab.research.google.com/github/shravanc/tf/blob/master/hin_to_eng.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#==================================Downloading===========================================

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
  
import urllib3
import shutil
import zipfile
import os
http = urllib3.PoolManager()

def extract(path, url, zipfilename):
  with http.request('GET', url, preload_content=False) as r, open(zipfilename, 'wb') as out_file:
    shutil.copyfileobj(r, out_file)

  print("file--->", zipfilename)
  with zipfile.ZipFile(zipfilename, 'r') as zip_ref:
    zip_ref.extractall(path)


# Dataset 1
url = 'https://github.com/shravanc/datasets/blob/master/hin_eng/hi-en.zip?raw=true'
filename = 'hi-en.zip'
path = os.getcwd()
zipfilename = os.path.join(path, filename)
extract(path, url, zipfilename)


# Dataset 2
url = 'https://github.com/shravanc/datasets/blob/master/hin_eng/parallel_corpora.zip?raw=true'
filename = 'parallel_corpora.zip'
path = os.getcwd()
zipfilename = os.path.join(path, filename)
extract(path, url, zipfilename)

#==================================Downloading===========================================

TensorFlow 2.x selected.


/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


file---> /content/hi-en.zip


/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


file---> /content/parallel_corpora.zip


In [ ]:
!ls hi-en/

dev.en	dev.hi	README	test.en  test.hi  train.en  train.hi


In [ ]:
!ls parallel_corpora/

_about.txt  hin.txt  README


**Feel free to check the data**

In [ ]:
!git clone https://github.com/shravanc/tf.git

fatal: destination path 'tf' already exists and is not an empty directory.


In [ ]:
!ls tf/hi_to_en

colab_lib  lib	main.py


In [ ]:
from tf.hi_to_en.colab_lib.utils import unicode_to_ascii, preprocess_sentence, create_dataset, create_new_dataset, load_dataset, max_length, convert
from tf.hi_to_en.colab_lib.encoder import Encoder
from tf.hi_to_en.colab_lib.attention import BahdanauAttention
from tf.hi_to_en.colab_lib.decoder import Decoder

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
  
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import io
import time
tf.executing_eagerly()
tf.__version__

'2.1.0'

In [ ]:
#==================================DataPreparation=======================================
# from lib.utils import unicode_to_ascii, preprocess_sentence, create_dataset, create_new_dataset, load_dataset, max_length, convert

en_sentence = u"May I borrow this book?"
sp_sentence = u"क्या मैं यह पुस्तक उधार ले सकता हूँ?"
print(preprocess_sentence(en_sentence))
print(preprocess_sentence(sp_sentence).encode('utf-8'))

path_to_file = os.path.join(os.getcwd(), "parallel_corpora/hin.txt")
en_1, hi_1 = create_dataset(path_to_file, None)

en_path = os.path.join(os.getcwd(), "hi-en/train.en")
hi_path = os.path.join(os.getcwd(), "hi-en/train.hi")
en_2, hi_2 = create_new_dataset(en_path, hi_path)

en = en_1 + en_2
hi = hi_1 + hi_2

# Try experimenting with the size of that dataset
num_examples = 80000
input_tensor, target_tensor, inp_lang, targ_lang = load_dataset(path_to_file, num_examples)

# Calculate max_length of the target tensors
max_length_targ, max_length_inp = max_length(target_tensor), max_length(input_tensor)


# Creating training and validation sets using an 80-20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.2)

# Show length
print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))

print ("Input Language; index to word mapping")
convert(inp_lang, input_tensor_train[0])
print ()
print ("Target Language; index to word mapping")
convert(targ_lang, target_tensor_train[0])



BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 64
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(inp_lang.word_index)+1
vocab_tar_size = len(targ_lang.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape
#==================================DataPreparation=======================================


<start> may i borrow this book ? <end>
b'<start> \xe0\xa4\x95\xe0\xa4\xaf\xe0\xa4\xbe \xe0\xa4\xae \xe0\xa4\xaf\xe0\xa4\xb9 \xe0\xa4\xaa\xe0\xa4\xb8\xe0\xa4\xa4\xe0\xa4\x95 \xe0\xa4\x89\xe0\xa4\xa7\xe0\xa4\xbe\xe0\xa4\xb0 \xe0\xa4\xb2 \xe0\xa4\xb8\xe0\xa4\x95\xe0\xa4\xa4\xe0\xa4\xbe \xe0\xa4\xb9 ? <end>'
2223 2223 556 556
Input Language; index to word mapping
1 ----> <start>
8 ----> वह
1534 ----> जलद
46 ----> ही
166 ----> ठीक
15 ----> हो
245 ----> जाएगा।
2 ----> <end>

Target Language; index to word mapping
1 ----> <start>
1328 ----> he'll
71 ----> get
137 ----> well
189 ----> soon
3 ----> .
2 ----> <end>


(TensorShape([64, 29]), TensorShape([64, 27]))

In [ ]:
#==================================Encoder===============================================
# from lib.encoder import Encoder
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

# sample input
sample_hidden = encoder.initialize_hidden_state()
sample_output, sample_hidden = encoder(example_input_batch, sample_hidden)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden.shape))

#==================================Encoder===============================================

Encoder output shape: (batch size, sequence length, units) (64, 29, 1024)
Encoder Hidden state shape: (batch size, units) (64, 1024)


In [ ]:
#==================================Attention=============================================
# from lib.attention import BahdanauAttention

attention_layer = BahdanauAttention(10)
attention_result, attention_weights = attention_layer(sample_hidden, sample_output)

print("Attention result shape: (batch size, units) {}".format(attention_result.shape))
print("Attention weights shape: (batch_size, sequence_length, 1) {}".format(attention_weights.shape))

#==================================Attention=============================================

Attention result shape: (batch size, units) (64, 1024)
Attention weights shape: (batch_size, sequence_length, 1) (64, 29, 1)


In [ ]:
#==================================Dencoder==============================================
# from lib.decoder import Decoder

decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

sample_decoder_output, _, _ = decoder(tf.random.uniform((BATCH_SIZE, 1)),
                                      sample_hidden, sample_output)

print ('Decoder output shape: (batch_size, vocab size) {}'.format(sample_decoder_output.shape))

#==================================Dencoder==============================================


Decoder output shape: (batch_size, vocab size) (64, 2389)


In [ ]:
#==================================Optimizer=============================================
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

#==================================Optimizer=============================================

In [ ]:
#==================================CheckPoint============================================
checkpoint_dir = './training_checkpoints_1'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

#==================================CheckPoint============================================


In [ ]:
#==================================Training==============================================
@tf.function
def train_step(inp, targ, enc_hidden):
  loss = 0

  with tf.GradientTape() as tape:
    enc_output, enc_hidden = encoder(inp, enc_hidden)

    dec_hidden = enc_hidden

    dec_input = tf.expand_dims([targ_lang.word_index['<start>']] * BATCH_SIZE, 1)

    # Teacher forcing - feeding the target as the next input
    for t in range(1, targ.shape[1]):
      # passing enc_output to the decoder
      predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)

      loss += loss_function(targ[:, t], predictions)

      # using teacher forcing
      dec_input = tf.expand_dims(targ[:, t], 1)

  batch_loss = (loss / int(targ.shape[1]))

  variables = encoder.trainable_variables + decoder.trainable_variables

  gradients = tape.gradient(loss, variables)

  optimizer.apply_gradients(zip(gradients, variables))

  return batch_loss
#==================================Training==============================================

In [ ]:
EPOCHS = 30

for epoch in range(EPOCHS):
  start = time.time()

  enc_hidden = encoder.initialize_hidden_state()
  total_loss = 0

  for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
    batch_loss = train_step(inp, targ, enc_hidden)
    total_loss += batch_loss

    if batch % 100 == 0:
      print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                   batch,
                                                   batch_loss.numpy()))
  # saving (checkpoint) the model every 2 epochs
  if (epoch + 1) % 2 == 0:
    checkpoint.save(file_prefix = checkpoint_prefix)

  print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))
  print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 0.3413
Epoch 1 Loss 0.3819
Time taken for 1 epoch 23.20542573928833 sec

Epoch 2 Batch 0 Loss 0.3277
Epoch 2 Loss 0.3394
Time taken for 1 epoch 7.379666805267334 sec

Epoch 3 Batch 0 Loss 0.2700
Epoch 3 Loss 0.2991
Time taken for 1 epoch 7.037050008773804 sec

Epoch 4 Batch 0 Loss 0.2328
Epoch 4 Loss 0.2568
Time taken for 1 epoch 7.561476230621338 sec

Epoch 5 Batch 0 Loss 0.1660
Epoch 5 Loss 0.2189
Time taken for 1 epoch 7.099998712539673 sec

Epoch 6 Batch 0 Loss 0.1560
Epoch 6 Loss 0.1807
Time taken for 1 epoch 7.611353397369385 sec

Epoch 7 Batch 0 Loss 0.1122
Epoch 7 Loss 0.1505
Time taken for 1 epoch 7.187697172164917 sec

Epoch 8 Batch 0 Loss 0.1084
Epoch 8 Loss 0.1265
Time taken for 1 epoch 7.62800407409668 sec

Epoch 9 Batch 0 Loss 0.1022
Epoch 9 Loss 0.1024
Time taken for 1 epoch 7.2221949100494385 sec

Epoch 10 Batch 0 Loss 0.0724
Epoch 10 Loss 0.0845
Time taken for 1 epoch 7.702613353729248 sec

Epoch 11 Batch 0 Loss 0.0718
Epoch 11 Loss 0.0670
Time tak

In [ ]:
#==================================Evaluate===========================================
def evaluate(sentence):
  attention_plot = np.zeros((max_length_targ, max_length_inp))

  sentence = preprocess_sentence(sentence)

  #inputs = [inp_lang.word_index[i] for i in sentence.split(' ')]
  inputs = []
  for i in sentence.split(' '):
    if i == "":
      break
    wi = inp_lang.word_index.get(i)
    print("index--->", i)
    print("class--->", type(inp_lang.word_index.get(i)))
    if wi:
      inputs.append(wi)


  inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                         maxlen=max_length_inp,
                                                         padding='post')
  inputs = tf.convert_to_tensor(inputs)

  result = ''

  hidden = [tf.zeros((1, units))]
  enc_out, enc_hidden = encoder(inputs, hidden)

  dec_hidden = enc_hidden
  dec_input = tf.expand_dims([targ_lang.word_index['<start>']], 0)

  for t in range(max_length_targ):
    predictions, dec_hidden, attention_weights = decoder(dec_input,
                                                         dec_hidden,
                                                         enc_out)

    # storing the attention weights to plot later on
    attention_weights = tf.reshape(attention_weights, (-1, ))
    attention_plot[t] = attention_weights.numpy()

    predicted_id = tf.argmax(predictions[0]).numpy()

    result += targ_lang.index_word[predicted_id] + ' '

    if targ_lang.index_word[predicted_id] == '<end>':
      return result, sentence, attention_plot

    # the predicted ID is fed back into the model
    dec_input = tf.expand_dims([predicted_id], 0)

  return result, sentence, attention_plot
#==================================Evaluate===========================================


In [ ]:
def translate(sentence):
  result, sentence, attention_plot = evaluate(sentence)

  print('Input: %s' % (sentence))
  print('Predicted translation: {}'.format(result))

  attention_plot = attention_plot[:len(result.split(' ')), :len(sentence.split(' '))]


In [ ]:
#==================================RestoreCheckPoint===================================
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))
#==================================RestoreCheckPoint===================================

In [ ]:
translate(u'देखने के लिए धन्यवाद')

index---> <start>
class---> <class 'int'>
index---> दखन
class---> <class 'int'>
index---> क
class---> <class 'int'>
index---> लिए
class---> <class 'int'>
index---> धनयवाद
class---> <class 'NoneType'>
index---> <end>
class---> <class 'int'>
Input: <start> दखन क लिए धनयवाद <end>
Predicted translation: you're a big like . <end> 
